콜백함수란? 어떤 함수를 수행 시 그 함수에서 사용자가 지정한 함수를 호출하는 것.

In [ ]:
""" ▶ Step 1. EarlyStopping을 테스트할 데이터 만들기 """

import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_circles
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

data, target = make_circles(n_samples=2000, noise=0.2, factor=0.3)
x_train, x_test, y_train, y_test = train_test_split(data, target, test_size=0.2, stratify=target)

fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(1, 1, 1, aspect='equal', xlim=(-2, 2), ylim=(-2, 2))
ax.scatter(x_train[y_train == 0, 0], x_train[y_train == 0, 1], marker="o")
ax.scatter(x_train[y_train == 1, 0], x_train[y_train == 1, 1], marker="x")
plt.show()

In [ ]:
""" ▶ Step 2. 간단한 신경망을 만들어서 학습시키자 """

model = Sequential([
    Dense(16, activation='tanh', input_shape=(2,)),
    Dense(1, activation='sigmoid'),
])
model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

hist = model.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=[x_test, y_test])

#Q4) 손실함수와 정답률의 변화를 시각화 
fig = plt.figure(figsize=(14, 6))
ax = fig.add_subplot(1, 2, 1, title="loss")
ax.plot(hist.epoch, hist.history["loss"], label="train_loss")
ax.plot(hist.epoch, hist.history["val_loss"], linestyle="-.", label="val_loss")
ax.legend()

ax = fig.add_subplot(1, 2, 2, title="acc")
ax.plot(hist.epoch, hist.history["acc"], label="train_acc")
ax.plot(hist.epoch, hist.history["val_acc"], linestyle="-.", label="val_acc")
ax.legend()
plt.show()

###  tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')
 
    값의 변화가 정지했을 때에 훈련을 종료하는 목적을 가진 메소드 


- monitor : 모니터링 할 값.
- min_delta : 모니터링 할 값에 대해 개선으로 결정되는 최소 변화 값. 즉, min_delta보다 절대치의 변화가 작으면 개선되어 있지 않다고 판단
- patience : 여기에 지정된 에포크 수 동안 (감시하는 값에) 개선이 없으면 훈련이 중지
- verbose : 중복 모드.
- mode : {auto, min, max} 중 하나가 선택. 
  - min : 모니터링 할 값의 감소가 중지되면 훈련이 종료
  - max : 모니터링 할 값의 증가가 중지되면 훈련이 종료. 
  - auto:  자동으로 모니터링되는 값

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

EarlyStopping(
    monitor='val_loss',   # 모니터링 지표값.
    min_delta=0,        # 최소 변화량    0.001
    patience=0,         # 지정된 에폭 수 동안 모니터링 지표의 개선이 없을 때, 학습을 중지. ex) 10을 주면 10에폭동안 개선이 없으면 중지됨.
    verbose=0,
    mode='auto',
    baseline=None,      # 지정된 수치값보다 개선되지 않으면 학습을 종료
    restore_best_weights=False,   # 마지막 학습된 가중치가 유지
    start_from_epoch=0     # early stop을 시작할 에폭 번호.
)

In [ ]:
""" ▶ Step 3. EarlyStopping을 넣어 학습시켜 보자 """
model = Sequential([
    Dense(16, activation='tanh', input_shape=(2,)),
    Dense(1, activation='sigmoid'),
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

early_stopping = EarlyStopping(monitor='val_loss',
                               min_delta=0.001,
                               patience=10, verbose=1, restore_best_weights=True)

hist = model.fit(x_train, y_train, epochs=100, batch_size=32, validation_data=[x_test, y_test],
                 callbacks=[early_stopping])
# monitor에 설정된 값이 patience 값의 횟수를 계속해서 min_delta 이상 개선되지 않으면 학습이 중지
              # monitor에는 'val_loss' 외에 'val_acc'등도 설정할 수 있다. 

In [ ]:
#Q4) 손실함수와 정답률의 변화를 시각화 
fig = plt.figure(figsize=(14, 6))
ax = fig.add_subplot(1, 2, 1, title="loss")
ax.plot(history.epoch, history.history["loss"], label="train_loss")
ax.plot(history.epoch, history.history["val_loss"], linestyle="-.", label="val_loss")
ax.legend()

ax = fig.add_subplot(1, 2, 2, title="acc")
ax.plot(history.epoch, history.history["acc"], label="train_acc")
ax.plot(history.epoch, history.history["val_acc"], linestyle="-.", label="val_acc")
ax.legend()
plt.show()

In [ ]:
# 정답이 특정값 이상됐을 떄 중단하고 싶으면. (90%)

from keras.callbacks import EarlyStopping


early_stopping =  EarlyStopping(
                            monitor='val_acc',
                            min_delta=0.0001,
                            patience=5, verbose=1, restore_best_weights=True,
                            mode = 'max'
)

history = model.fit(
                    X_train,
                    y_train,
                    epochs=100,
                    batch_size=16,
                    validation_data=[X_test, y_test],
                    callbacks=[early_stopping] 
            )# monitor에 설정된 값이 patience 값의 횟수를 계속해서 min_delta 이상 개선되지 않으면 학습이 중지
              # monitor에는 'val_loss' 외에 'val_acc'등도 설정할 수 있다. 